<a href="https://colab.research.google.com/github/sofial47/senalesysistemas/blob/main/parcial2dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#instalación de librerías
!pip install streamlit -q

In [2]:
!pip install ipywidgets matplotlib numpy scipy
#Comando de terminal. Sirve para instalar librerías externas necesarias para ejecutar la simulación interactiva
#ipywidgets: Permite crear interfaces gráficas interactivas en notebooks, como sliders, checkboxes, botones, etc. Muy útil para que el usuario pueda elegir parámetros como ζ, ωn, etc
#matplotlib: Librería para graficar.Sirve para mostrar cómo cambia la posición, velocidad, aceleración, etc., del sistema en función del tiempo.
#numpy: Librería para trabajar con arreglos numéricos, operaciones vectoriales y matemáticas avanzadas
#scipy: Contiene funciones científicas, entre ellas solve_ivp, que permite resolver ecuaciones diferenciales.
!pip install control matplotlib numpy scipy
!pip install control ipywidgets --quiet
##!pip uninstall control -y
##!pip install control --upgrade
#!pip install control


In [3]:
##!pip uninstall control -y
!pip install streamlit yt-dlp pydub numpy matplotlib scipy
!apt-get install ffmpeg -y
!pip install yt-dlp pydub scipy matplotlib numpy

!pip install control==0.10.2 --no-cache-dir --force-reinstall
!pip install numpy matplotlib scipy ipywidgets pydub yt_dlp ffmpeg-python --quiet


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 133.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.3/578.3 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 186.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 173.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 206.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.2/326.2 kB 162.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 259.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 M

In [4]:
##!pip uninstall -y control


In [5]:
#!pip install control==0.10.2 --no-cache-dir --force-reinstall


In [6]:
!mkdir pages

In [7]:
%%writefile 0_👋_Dshboard.py

import streamlit as st

st.set_page_config(
    page_title="Bienvenida",
    page_icon="👋",
)

st.write("# Bienvenido a mi Dshboard! 👋")

st.sidebar.success("Seleccciona una demo a explorar.")

#st.markdown(
   # """
    #Streamlit es una aplicación de código abierto creado específicamente para
   # Proyectos de Machine Learning y Data Science.
    #**👈 Seleccione una demostración de la barra lateral** para ver algunos ejemplos
    #¡De lo que Streamlit puede hacer!
    ### ¿Quieres saber más?
    #- Consulta [streamlit.io] (https://streamlit.io)
    #- Revisa la [documentación](https://docs.streamlit.io)
#"""
#)

Writing 0_👋_Dshboard.py


In [8]:
%%writefile 1_📈_Plotting_Demo.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Configuración inicial del panel
st.set_page_config(page_title="Simulación Masa-Resorte-Amortiguador", page_icon="🌀")
st.title("🌀 Simulación Sistema Masa-Resorte-Amortiguador")

# ---- Entradas de usuario ----
tipo = st.selectbox("Seleccione el tipo de respuesta:",
                    ['Subamortiguada', 'Crítico', 'Sobreamortiguada', 'Inestable'])

# Ajustes según el tipo
if tipo == 'Subamortiguada':
    zeta = st.slider("ζ (factor de amortiguamiento)", 0.01, 0.99, 0.3, 0.01)
elif tipo == 'Crítico':
    zeta = 1.0
    st.write("ζ (factor de amortiguamiento): 1.0 (Crítico)")
elif tipo == 'Sobreamortiguada':
    zeta = st.slider("ζ (factor de amortiguamiento)", 1.01, 3.0, 1.5, 0.01)
elif tipo == 'Inestable':
    zeta = st.slider("ζ (factor de amortiguamiento)", -2.0, -0.01, -0.3, 0.01)

wn = st.slider("ωₙ (frecuencia natural)", 1.0, 30.0, 10.0, 0.5)
t_max = st.slider("Tiempo de simulación (s)", 1, 20, 10)

# ---- Función del sistema ----
def sistema(t, y, zeta, wn, F=1.0, m=1.0):
    return [y[1], -2*zeta*wn*y[1] - wn**2*y[0] + F/m]

# ---- Simulación ----
y0 = [0, 0]
t_eval = np.linspace(0, t_max, 1000)
sol = solve_ivp(sistema, [0, t_max], y0, t_eval=t_eval, args=(zeta, wn))

# ---- Gráfica ----
fig, ax = plt.subplots()
ax.plot(sol.t, sol.y[0], label=f"ζ = {zeta}, ωₙ = {wn}")
ax.set_title("Desplazamiento vs Tiempo")
ax.set_xlabel("Tiempo (s)")
ax.set_ylabel("Desplazamiento y(t)")
ax.grid(True)
ax.legend()

st.pyplot(fig)


Writing 1_📈_Plotting_Demo.py


In [9]:
!mv 1_📈_Plotting_Demo.py pages/

In [10]:
#!pip uninstall -y control


In [11]:
#import control
#print(control.__version__)
#print(control.__file__)


In [12]:
#import control
#print(control.__version__)
#print(hasattr(control, "pole"))


In [13]:
# 2_🌍_Mapping_Demo.py
%%writefile 2_🌍_Mapping_Demo.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import control as ctl


st.set_page_config(page_title="Simulación Sistema Masa-Resorte-Amortiguador", page_icon="🌍")
st.title("🌍 Simulación de Sistemas Dinámicos")

# --- Entradas del usuario ---
st.sidebar.header("Parámetros del sistema")

zeta = st.sidebar.slider('ζ (Amortiguamiento)', min_value=0.01, max_value=1.5, value=0.3, step=0.01)
wn = st.sidebar.slider('ωₙ (Frecuencia natural)', min_value=0.5, max_value=10.0, value=2.0, step=0.1)
masa = st.sidebar.slider('Masa (kg)', min_value=0.1, max_value=10.0, value=1.0, step=0.1)
t_max = st.sidebar.slider('Tiempo de simulación (s)', min_value=5, max_value=30, value=10, step=1)

# --- Modelo ---
k = wn**2 * masa
c = 2 * zeta * np.sqrt(k * masa)
G_abierto = ctl.tf([wn**2], [1, 2*zeta*wn, wn**2])
G_cerrado = ctl.feedback(G_abierto, 1)

t = np.linspace(0, t_max, 1000)
rampa = t

# --- Respuestas ---
_, y_imp = ctl.impulse_response(G_abierto, T=t)
_, y_imp_c = ctl.impulse_response(G_cerrado, T=t)
_, y_step = ctl.step_response(G_abierto, T=t)
_, y_step_c = ctl.step_response(G_cerrado, T=t)
_, y_rampa = ctl.forced_response(G_abierto, T=t, U=rampa)
_, y_rampa_c = ctl.forced_response(G_cerrado, T=t, U=rampa)

# --- Gráficas ---
st.subheader("📉 Respuestas del Sistema")

fig, axs = plt.subplots(3, 2, figsize=(14, 9))
axs[0, 0].plot(t, y_imp); axs[0, 0].set_title("Impulso - Lazo Abierto"); axs[0, 0].grid()
axs[0, 1].plot(t, y_imp_c); axs[0, 1].set_title("Impulso - Lazo Cerrado"); axs[0, 1].grid()
axs[1, 0].plot(t, y_step); axs[1, 0].set_title("Escalón - Lazo Abierto"); axs[1, 0].grid()
axs[1, 1].plot(t, y_step_c); axs[1, 1].set_title("Escalón - Lazo Cerrado"); axs[1, 1].grid()
axs[2, 0].plot(t, y_rampa); axs[2, 0].set_title("Rampa - Lazo Abierto"); axs[2, 0].grid()
axs[2, 1].plot(t, y_rampa_c); axs[2, 1].set_title("Rampa - Lazo Cerrado"); axs[2, 1].grid()
st.pyplot(fig)

# --- Parámetros físicos y eléctricos ---
st.subheader("⚙️ Parámetros físicos y eléctricos equivalentes")

col1, col2 = st.columns(2)

with col1:
    st.markdown("**🔧 Sistema Mecánico:**")
    st.write(f"Masa (m): `{masa:.3f} kg`")
    st.write(f"Constante del resorte (k): `{k:.3f} N/m`")
    st.write(f"Coef. de amortiguamiento (c): `{c:.3f} Ns/m`")

with col2:
    st.markdown("**⚡ Sistema Eléctrico Equivalente (Paralelo):**")
    st.write(f"Inductancia (L): `{masa:.3f} H`")
    st.write(f"Resistencia (R): `{c:.3f} Ω`")
    st.write(f"Capacitancia (C): `{1/k:.6f} F`")

# --- Parámetros temporales ---
st.subheader("📊 Parámetros Temporales (Respuesta Escalón)")

info_open = ctl.step_info(G_abierto)
info_closed = ctl.step_info(G_cerrado)

st.write("🔓 **Lazo Abierto**")
for k, v in info_open.items():
    st.write(f" - {k}: `{v:.4f}`")

st.write("🔒 **Lazo Cerrado**")
for k, v in info_closed.items():
    st.write(f" - {k}: `{v:.4f}`")
# --- Diagrama de Bode ---
st.subheader("📡 Diagrama de Bode")

#mag_open, phase_open, omega = ctl.bode(G_abierto, Plot=False)
#mag_closed, phase_closed, _ = ctl.bode(G_cerrado, Plot=False)
#mag_open, phase_open, omega = ctl.bode(G_abierto, plot=False)
#mag_closed, phase_closed, _ = ctl.bode(G_cerrado, plot=False)
omega = np.logspace(-1, 2, 500)  # de 0.1 a 100 rad/s

mag_open, phase_open, _ = ctl.bode(G_abierto, omega, plot=False)
mag_closed, phase_closed, _ = ctl.bode(G_cerrado, omega, plot=False)



fig_bode, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(10, 6))
ax_mag.semilogx(omega, 20 * np.log10(mag_open), label="Abierto")
ax_mag.semilogx(omega, 20 * np.log10(mag_closed), label="Cerrado")
ax_mag.set_ylabel("Magnitud (dB)")
ax_mag.set_title("Bode - Magnitud")
ax_mag.legend()
ax_mag.grid(True)

ax_phase.semilogx(omega, np.degrees(phase_open), label="Abierto")
ax_phase.semilogx(omega, np.degrees(phase_closed), label="Cerrado")
ax_phase.set_ylabel("Fase (°)")
ax_phase.set_xlabel("Frecuencia (rad/s)")
ax_phase.set_title("Bode - Fase")
ax_phase.legend()
ax_phase.grid(True)

st.pyplot(fig_bode)





# --- Polos y Ceros ---
st.subheader("🌀 Mapa de Polos y Ceros")

fig_pz, ax_pz = plt.subplots(figsize=(3, 3))

# Manual calculation of poles and zeros
# G_abierto = ctl.tf([wn**2], [1, 2*zeta*wn, wn**2])
# Numerator coefficients for open loop: [wn**2]
# Denominator coefficients for open loop: [1, 2*zeta*wn, wn**2]
poles_ab = np.roots([1, 2*zeta*wn, wn**2])
zeros_ab = np.roots([wn**2]) # This will be an empty array if wn is non-zero

# G_cerrado = ctl.feedback(G_abierto, 1)
# For closed loop G_cl = G_abierto / (1 + G_abierto)
# G_cl = (wn**2 / (s^2 + 2*zeta*wn*s + wn**2)) / (1 + wn**2 / (s^2 + 2*zeta*wn*s + wn**2))
# G_cl = wn**2 / (s^2 + 2*zeta*wn*s + wn**2 + wn**2)
# G_cl = wn**2 / (s^2 + 2*zeta*wn*s + 2*wn**2)
# Numerator coefficients for closed loop: [wn**2]
# Denominator coefficients for closed loop: [1, 2*zeta*wn, 2*wn**2]

poles_cl = np.roots([1, 2*zeta*wn, 2*wn**2])
zeros_cl = np.roots([wn**2]) # Same zeros as open loop


# 🎯 Dibujar polos y ceros
ax_pz.plot(np.real(poles_ab), np.imag(poles_ab), 'rx', label='Polos (Abierto)')
# Only plot zeros if there are any (i.e., if wn is zero)
if zeros_ab.size > 0:
    ax_pz.plot(np.real(zeros_ab), np.imag(zeros_ab), 'bo', label='Ceros (Abierto)')

ax_pz.plot(np.real(poles_cl), np.imag(poles_cl), 'r*', label='Polos (Cerrado)')
# Only plot zeros if there are any (i.e., if wn is zero)
if zeros_cl.size > 0:
    ax_pz.plot(np.real(zeros_cl), np.imag(zeros_cl), 'b^', label='Ceros (Cerrado)')


# 🧭 Decoración
ax_pz.set_xlabel('Parte Real')
ax_pz.set_ylabel('Parte Imaginaria')
ax_pz.set_title('Mapa de Polos y Ceros')
ax_pz.grid(True)
ax_pz.axhline(0, color='black', linewidth=0.5)
ax_pz.axvline(0, color='black', linewidth=0.5)
ax_pz.legend()
ax_pz.axis('equal')

# Mostrar en Streamlit
st.pyplot(fig_pz)

Writing 2_🌍_Mapping_Demo.py


In [14]:
!mv 2_🌍_Mapping_Demo.py pages/

In [15]:
#import control
#print(control.__version__)
#print(hasattr(control, "pole"))


In [16]:
%%writefile 3_📊_DataFrame_Demo.py
# -*- coding: utf-8 -*-
# 3_📊_DataFrame_Demo.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert, butter, filtfilt
from scipy.io import wavfile
# No necesitamos IPython.display.Audio ni display() en Streamlit
from pydub import AudioSegment
import yt_dlp
import os
import tempfile
import shutil # Importar shutil para limpiar directorios temporales

# ===========================
# 🔧 CONFIGURACIÓN INICIAL
# ===========================
st.set_page_config(page_title="SSB-AM Dashboard", layout="wide")
st.title("📡 Modulación y Demodulación SSB-AM")

st.write("Script 3_📊_DataFrame_Demo.py iniciado.") # Mensaje de depuración


# ====================================
# 🎵 FUNCIÓN PARA CARGAR AUDIO DE YOUTUBE
# ====================================
@st.cache_data # Cachear la función para evitar descargas repetidas
def cargar_audio_youtube(url, duracion=5, fs_objetivo=44100):
    st.write(f"Intentando descargar audio de: {url}") # Mensaje de depuración
    temp_dir = None # Inicializar a None
    try:
        temp_dir = tempfile.mkdtemp()
        salida_wav = os.path.join(temp_dir, "audio.wav")

        # Configuración de descarga con yt_dlp
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(temp_dir, "audio.%(ext)s"),
            'quiet': True,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
                'preferredquality': '192',
            }],
            'nocheckcertificate': True,
            'logtostderr': False,
            'no_warnings': True,
            'default_search': 'ytsearch',
            # 'max_downloads': 1, # Limitar a una descarga
            'extract_flat': 'discard_key', # Descargar directamente, no listar
        }

        # Descargar desde YouTube
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            # Buscar el archivo wav generado por el postprocessor
            downloaded_file = None
            if 'requested_downloads' in info_dict:
                 for dl in info_dict['requested_downloads']:
                     if dl['filepath'].endswith('.wav'):
                         downloaded_file = dl['filepath']
                         break
            if downloaded_file is None:
                 # Fallback: buscar cualquier .wav en el temp_dir
                 wav_files = [f for f in os.listdir(temp_dir) if f.endswith('.wav')]
                 if wav_files:
                      downloaded_file = os.path.join(temp_dir, wav_files[0])


            if downloaded_file and os.path.exists(downloaded_file):
                 salida_wav = downloaded_file
            else:
                 st.error("Error: No se pudo encontrar el archivo WAV descargado.")
                 return None, None


        # Leer audio
        audio = AudioSegment.from_file(salida_wav)
        audio = audio.set_channels(1).set_frame_rate(fs_objetivo)
        audio = audio[:duracion * 1000]  # Cortar a 5 segundos
        # No necesitamos exportar a un archivo temporal para wavfile.read si AudioSegment puede dar los datos
        # Pero wavfile.read es más seguro para obtener fs y data_np
        audio.export(salida_wav, format="wav") # Re-exportar para wavfile.read compatibilidad


        # Leer audio como arreglo NumPy
        fs, data = wavfile.read(salida_wav)
        # Asegurarse de que los datos sean de tipo float
        if data.dtype != np.float32 and data.dtype != np.float64:
             data = data.astype(np.float32)

        # Normalización
        max_abs_data = np.max(np.abs(data))
        if max_abs_data > 0:
             data = data / max_abs_data
        else:
             data = np.zeros_like(data)


        st.write("Audio descargado y procesado correctamente.") # Mensaje de depuración
        return fs, data

    except Exception as e:
        st.error(f"Error durante la descarga o procesamiento del audio de YouTube: {e}")
        return None, None
    finally:
        # Limpiar archivos temporales
        if temp_dir and os.path.exists(temp_dir):
            st.write(f"Limpiando directorio temporal: {temp_dir}") # Mensaje de depuración
            #shutil.rmtree(temp_dir) # Comentar por ahora para depuración si es necesario
            pass # No limpiar inmediatamente para inspección si hay errores

# ====================================
# 📡 FUNCIÓN DE MODULACIÓN Y DEMODULACIÓN SSB
# ====================================
# No necesitamos cachear esta función ya que sus entradas cambian con los sliders
def ssb_mod_demod(m, fs, fc=10000):
    st.write(f"Iniciando modulación/demodulación SSB con fc={fc} Hz.") # Mensaje de depuración
    if m is None or fs is None or len(m) == 0:
        st.warning("No se pudo procesar la señal de entrada para SSB.")
        # Retornar valores que no causen errores en el graficado/audio posterior
        t = np.linspace(0, 1, 100) # Tiempo de ejemplo
        zeros = np.zeros_like(t, dtype=np.float32) # Asegurar dtype flotante
        return t, zeros, zeros, zeros

    try:
        t = np.arange(0, len(m)) / fs
        m = m - np.mean(m)  # Centrar señal

        # Modulación SSB
        st.write("Calculando transformada de Hilbert y modulando.") # Mensaje de depuración
        m_hilbert = np.imag(hilbert(m))  # Transformada de Hilbert
        ssb = m * np.cos(2 * np.pi * fc * t) - m_hilbert * np.sin(2 * np.pi * fc * t)

        # Demodulación coherente
        st.write("Demodulando y filtrando.") # Mensaje de depuración
        demod = ssb * np.cos(2 * np.pi * fc * t)

        # Filtro pasa bajos
        nyquist = 0.5 * fs
        normal_cutoff = fc / nyquist
        if normal_cutoff >= 1: # Usar >= para ser seguro
            st.warning(f"La frecuencia de corte del filtro ({fc} Hz) es >= la frecuencia de Nyquist ({nyquist} Hz). Ajustando a 0.99 * Nyquist.")
            normal_cutoff = 0.99 # Ajuste para evitar problemas con butter/filtfilt en el límite


        # Asegurarse de que normal_cutoff sea positivo y válido
        if normal_cutoff <= 0:
             st.error(f"Frecuencia de corte del filtro no válida: {fc}. Usando un valor mínimo.")
             normal_cutoff = 0.01 # Valor mínimo seguro

        b, a = butter(5, normal_cutoff, btype='low', analog=False) # Asegurar digital filter
        m_rec = filtfilt(b, a, demod)

        st.write("Procesamiento SSB completado.") # Mensaje de depuración
        return t, m, ssb, m_rec

    except Exception as e:
        st.error(f"Error durante la modulación o demodulación SSB: {e}")
        t = np.arange(0, len(m)) / fs if len(m) > 0 else np.linspace(0, 1, 100)
        zeros = np.zeros_like(t, dtype=np.float32)
        return t, m, zeros, zeros


# ====================================
# 🎚 INTERFAZ DE USUARIO (Streamlit Sidebar)
# ====================================
st.sidebar.header("⚙️ Parámetros")

modo = st.sidebar.selectbox("Fuente de señal mensaje:", ["Pulso rectangular", "Audio de YouTube"])

url_youtube_default = "https://www.youtube.com/watch?v=0LTg8gYVFxM"
url_youtube = ""
if modo == "Audio de YouTube":
     url_youtube = st.sidebar.text_input("URL de YouTube", value=url_youtube_default)


fc = st.sidebar.slider("Frecuencia portadora (Hz)", 1000, 20000, 10000, step=1000)


# ====================================
# Entrada de señal (Basado en selección del usuario)
# ====================================
m = None
fs = None

if modo == "Pulso rectangular":
    fs = 44100
    # Definir duración y frecuencia del pulso si se quiere hacer ajustable
    duracion_pulso = st.sidebar.slider("Duración del pulso (s)", 0.01, 0.1, 0.05, 0.005)
    t_pulso = np.linspace(0, duracion_pulso, int(fs * duracion_pulso))
    # Definir los límites del pulso dentro de t_pulso
    inicio_pulso = duracion_pulso / 4
    fin_pulso = 3 * duracion_pulso / 4
    m = np.where((t_pulso >= inicio_pulso) & (t_pulso <= fin_pulso), 1.0, 0.0)
    t = t_pulso # Usar el tiempo del pulso para graficar
    st.write(f"Modo: Pulso rectangular generado. Duración: {len(t)/fs:.4f}s") # Mensaje de depuración


elif modo == "Audio de YouTube" and url_youtube:
    fs, m = cargar_audio_youtube(url_youtube)
    # Si se carga correctamente, generar el vector de tiempo
    if m is not None and fs is not None:
        t = np.arange(0, len(m)) / fs
        st.write(f"Audio de YouTube cargado. Duración: {len(m)/fs:.4f}s") # Mensaje de depuración
    else:
         st.error("No se pudo cargar el audio de YouTube. Por favor, verifica la URL.")
         t = np.linspace(0, 1, 100) # Tiempo de ejemplo para evitar errores si m es None
         m = np.zeros_like(t, dtype=np.float32) # Señal vacía para evitar errores


# ====================================
# Procesamiento y Visualización (Si la señal es válida)
# ====================================
if m is not None and fs is not None and len(m) > 0:
    st.write("Procesando y mostrando resultados.") # Mensaje de depuración

    t, m_orig, ssb, m_rec = ssb_mod_demod(m, fs, fc)

    # 📊 GRÁFICAS
    st.subheader("📉 Gráficas de Señales (Dominio del Tiempo)")

    try:
        fig, axs = plt.subplots(3, 1, figsize=(10, 6))

        axs[0].plot(t, m_orig)
        axs[0].set_title("🔹 Señal mensaje original")
        axs[0].set_ylabel("Amplitud")
        axs[0].grid(True)

        axs[1].plot(t, ssb)
        axs[1].set_title("🔸 Señal modulada SSB")
        axs[1].set_ylabel("Amplitud")
        axs[1].grid(True)

        axs[2].plot(t, m_rec)
        axs[2].set_title("🔻 Señal demodulada (recuperada)")
        axs[2].set_xlabel("Tiempo (s)")
        axs[2].set_ylabel("Amplitud")
        axs[2].grid(True)

        plt.tight_layout()
        st.pyplot(fig)
        st.write("Gráficas generadas y mostradas.") # Mensaje de depuración
    except Exception as e:
        st.error(f"Error al generar o mostrar gráficas: {e}")


    # 🔊 AUDIOS
    st.subheader("🔊 Audios")

    try:
        st.write("Reproduciendo audios.") # Mensaje de depuración
        # Streamlit usa st.audio() directamente con los datos o la ruta del archivo
        # No necesitas IPython.display.Audio ni display() aquí
        st.write("Señal original:")
        st.audio(m_orig, sample_rate=fs, format='audio/wav', start_time=0)

        st.write("Señal modulada (SSB):") # Nuevo reproductor para la señal modulada
        st.audio(ssb, sample_rate=fs, format='audio/wav', start_time=0)


        st.write("Señal demodulada (recuperada):")
        st.audio(m_rec, sample_rate=fs, format='audio/wav', start_time=0)
        st.write("Audios generados y mostrados.") # Mensaje de depuración
    except Exception as e:
        st.error(f"Error al generar o mostrar audios: {e}")


else:
    st.write("Esperando selección o carga de señal mensaje...") # Mensaje de depuración

st.write("Script 3_📊_DataFrame_Demo.py finalizado.") # Mensaje de depuración

Writing 3_📊_DataFrame_Demo.py


In [17]:
!mv 3_📊_DataFrame_Demo.py pages/

In [18]:
#!cat /content/logs.txt

In [19]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 0_👋_Dshboard.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-07-15 01:29:16--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64 [following]
--2025-07-15 01:29:16--  https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/37d2bad8-a2ed-4b93-8139-cbb15162d81d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-15T02%3A03%3A53Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-15T0

In [20]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")


Digite (1) para finalizar la ejecución del Dashboard: 1
El proceso de Streamlit ha sido finalizado.
